In [1]:
import deeplabcut
import os
import imageio
import imageio.v3 as iio
import numpy as np
import pandas as pd
import pathlib
import re
import glob
import pandas as pd

Loading DLC 3.0.0rc10...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


c:\Users\Rober\anaconda3\envs\DEEPLABCUT\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
def get_bodyparts_from_xma(csv_path: str,
                            mode: str):
    """Takes the filepath of an XMAlab CSV file and returns marker names"""

    trial_csv = pd.read_csv(
        csv_path,
        sep=",",
        header=0,
        dtype="float",
        na_values="NaN",
    )
    names = trial_csv.columns.values
    if mode == "rgb":
        parts = [name.rsplit("_", 1)[0] for name in names]
    elif mode in ["2D", "per_cam"]:
        parts = [name.rsplit("_", 2)[0] for name in names]
    else:
        raise SyntaxError("Invalid value for mode parameter")

    # I do it this way to maintain ordering in the list, since that's
    # important for DeepLabCut
    parts_unique = []
    for part in parts:
        if part not in parts_unique:
            parts_unique.append(part)
    return parts_unique

def _splice_xma_to_dlc(self, trial_path, outlier_mode=False):
    """Takes csv of XMALab 2D XY coordinates from 2 cameras, outputs spliced hdf+csv data for DeepLabCut"""
    substitute_data_relpath = "labeled-data/" + self._config["dataset_name"]
    substitute_data_abspath = os.path.join(
        os.path.sep.join(self._config["path_config_file"].split("\\")[:-1]),
        substitute_data_relpath,
    )
    trial_csv_path = self.find_trial_csv(trial_path)
    markers = self.get_bodyparts_from_xma(trial_csv_path, mode='2D')

    # TODO: this entire section can be solved with a creative call to
    # get_bodyparts_from_xma and some dataFrame manipulation to be
    # significantly shorter (and potentially faster?)
    try:
        trial_name = os.path.basename(os.path.normpath(trial_path))
        df = pd.read_csv(f"{trial_path}/{trial_name}.csv")
    except FileNotFoundError as e:
        raise FileNotFoundError(
            f"Please make sure that your trainingdata 2DPoints csv file is named {trial_name}.csv"
        ) from e
    if self._swap_markers:
        print("Creating cam1Y-cam2Y-swapped synthetic markers")
        swaps = []
        df_sw = pd.DataFrame()
        for marker in markers:
            name_x1 = marker + "_cam1_X"
            name_x2 = marker + "_cam2_X"
            name_y1 = marker + "_cam1_Y"
            name_y2 = marker + "_cam2_Y"
            swap_name_x1 = "sw_" + name_x1
            swap_name_x2 = "sw_" + name_x2
            swap_name_y1 = "sw_" + name_y1
            swap_name_y2 = "sw_" + name_y2
            df_sw[swap_name_x1] = df[name_x1]
            df_sw[swap_name_y1] = df[name_y2]
            df_sw[swap_name_x2] = df[name_x2]
            df_sw[swap_name_y2] = df[name_y1]
            swaps.extend([swap_name_x1, swap_name_y1, swap_name_x2, swap_name_y2])
        df = df.join(df_sw)
        print(swaps)
    if self._cross_markers:
        print("Creating cam1-cam2-crossed synthetic markers")
        crosses = []
        df_cx = pd.DataFrame()
        for marker in markers:
            name_x1 = marker + "_cam1_X"
            name_x2 = marker + "_cam2_X"
            name_y1 = marker + "_cam1_Y"
            name_y2 = marker + "_cam2_Y"
            cross_name_x = "cx_" + marker + "_cam1x2_X"
            cross_name_y = "cx_" + marker + "_cam1x2_Y"
            df_cx[cross_name_x] = df[name_x1] * df[name_x2]
            df_cx[cross_name_y] = df[name_y1] * df[name_y2]
            crosses.extend([cross_name_x, cross_name_y])
        df = df.join(df_cx)
        print(crosses)
    names_final = df.columns.values
    parts_final = [name.rsplit("_", 1)[0] for name in names_final]
    parts_unique_final = []
    for part in parts_final:
        if not part in parts_unique_final:
            parts_unique_final.append(part)
    print("Importing markers: ")
    print(parts_unique_final)
    with open(self._config["path_config_file"], "r") as dlc_config:
        yaml = YAML()
        dlc_proj = yaml.load(dlc_config)

    dlc_proj["bodyparts"] = parts_unique_final

    with open(self._config["path_config_file"], "w") as dlc_config:
        yaml.dump(dlc_proj, dlc_config)

    df = df.dropna(how="all")
    list_of_frames = df.index + 1
    unique_frames_set = set(list_of_frames)
    unique_frames = sorted(unique_frames_set)
    print("Importing frames: ")
    print(unique_frames)
    df["frame_index"] = [
        substitute_data_relpath
        + f"/{trial_name}_rgb_"
        + str(index).zfill(4)
        + ".png"
        for index in unique_frames
    ]
    df["scorer"] = self._config["experimenter"]
    df = df.melt(id_vars=["frame_index", "scorer"])
    new = df["variable"].str.rsplit("_", n=1, expand=True)
    df["variable"], df["coords"] = new[0], new[1]
    df = df.rename(columns={"variable": "bodyparts"})
    df["coords"] = df["coords"].str.rstrip(" ").str.lower()
    cat_type = pd.api.types.CategoricalDtype(
        categories=parts_unique_final, ordered=True
    )
    df["bodyparts"] = df["bodyparts"].str.lstrip(" ").astype(cat_type)
    newdf = df.pivot_table(
        columns=["scorer", "bodyparts", "coords"],
        index="frame_index",
        values="value",
        aggfunc="first",
        dropna=False,
    )
    newdf.index.name = None
    if not os.path.exists(substitute_data_abspath):
        os.makedirs(substitute_data_abspath)
    if outlier_mode:
        data_name = os.path.join(substitute_data_abspath, "MachineLabelsRefine.h5")
        tracked_hdf = os.path.join(
            substitute_data_abspath, ("MachineLabelsRefine_" + ".h5")
        )
    else:
        data_name = os.path.join(
            substitute_data_abspath,
            ("CollectedData_" + self._config["experimenter"] + ".h5"),
        )
        tracked_hdf = os.path.join(
            substitute_data_abspath,
            ("CollectedData_" + self._config["experimenter"] + ".h5"),
        )
    newdf.to_hdf(data_name, "df_with_missing", format="table", mode="w")
    newdf.to_hdf(tracked_hdf, "df_with_missing", format="table", mode="w")
    tracked_csv = data_name.split(".h5")[0] + ".csv"
    newdf.to_csv(tracked_csv, na_rep="NaN")
    print(
        "Successfully spliced XMALab 2D points to DLC format",
        "saved " + str(data_name),
        "saved " + str(tracked_hdf),
        "saved " + str(tracked_csv),
        sep="\n",
    )

import cv2
import numpy as np

def imagej_auto_contrast(img, saturated=0.35):
    """
    Emulates ImageJ Auto-Adjust Brightness/Contrast.
    img: grayscale, numpy array (uint8 or uint16)
    saturated: percentage of pixels to clip at each end (default 0.35%)
    Returns contrast-stretched uint8 image.
    """
    img = img.astype(np.float32)
    low, high = np.percentile(img, (saturated, 100.0 - saturated))
    if high - low < 1e-3:
        stretched = np.clip(img, 0, 255)
    else:
        stretched = (img - low) * 255.0 / (high - low)
        stretched = np.clip(stretched, 0, 255)
    return stretched.astype(np.uint8)

def merge_avi_to_rgb(avi1, avi2, out_path, mode="difference", saturated=0.35, codec="XVID", fps=None):
    cap1 = cv2.VideoCapture(avi1)
    cap2 = cv2.VideoCapture(avi2)
    w = int(cap1.get(cv2.CAP_PROP_FRAME_WIDTH))
    h = int(cap1.get(cv2.CAP_PROP_FRAME_HEIGHT))
    if fps is None:
        fps = cap1.get(cv2.CAP_PROP_FPS) or 30

    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(out_path, fourcc, fps, (w, h))

    frame_num = 0
    while True:
        ret1, frame1 = cap1.read()
        ret2, frame2 = cap2.read()
        if not (ret1 and ret2):
            break
        gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        # --- Apply ImageJ auto contrast normalization ---
        norm1 = imagej_auto_contrast(gray1, saturated)
        norm2 = imagej_auto_contrast(gray2, saturated)
        # Use normalized frames for blue channel and output
        if mode == "difference":
            blue = cv2.absdiff(norm1, norm2)
        elif mode == "multiply":
            blue = ((norm1.astype(np.uint16) * norm2.astype(np.uint16)) // 255).astype(np.uint8)
        else:
            blue = np.zeros_like(norm1, dtype=np.uint8)
        merged = cv2.merge([blue, norm2, norm1])   # OpenCV expects BGR image order
        out.write(merged)
        frame_num += 1
        if frame_num % 50 == 0:
            print(f"Processed {frame_num} frames", end='\r')
    cap1.release()
    cap2.release()
    out.release()
    print(f"\nRGB video created: {out_path}")


In [ ]:
# Example usage:
avi1 = r"F:\In Vivo XROMM EMG\28MAY2025\Trials AVIs\28MAY2025_raisin_trial4_rat101_C001H001S0001.avi"
avi2 = r"F:\In Vivo XROMM EMG\28MAY2025\Trials AVIs\28MAY2025_raisin_trial4_rat101_C002H001S0001.avi"
out_video = r'F:\In Vivo XROMM EMG\28MAY2025\28MAY2025_raisin_trial4_rat101_RGB_output2.avi'
merge_avi_to_rgb(avi1, avi2, out_video, mode="difference", saturated=0.35, codec="MJPG")

In [ ]:
reader1 = iio.imiter(r"F:\In Vivo XROMM EMG\28MAY2025\Trials AVIs\28MAY2025_almond_trial21_rat101_C001H001S0001.avi")  # your actual files
reader2 = iio.imiter(r"F:\In Vivo XROMM EMG\28MAY2025\Trials AVIs\28MAY2025_almond_trial21_rat101_C002H001S0001.avi")
writer = imageio.get_writer(r'F:\In Vivo XROMM EMG\28MAY2025\28MAY2025_almond_trial21_rat101_RGB_output2.avi', fps=30)

for f1, f2 in zip(reader1, reader2):
    
    f1_gray = (0.114*f1[:,:,0] + 0.587*f1[:,:,1] + 0.299*f1[:,:,2]).astype(np.uint8)
    f1_gray = imagej_auto_contrast(f1_gray, 0.35)
    f2_gray = (0.114*f2[:,:,0] + 0.587*f2[:,:,1] + 0.299*f2[:,:,2]).astype(np.uint8)
    f2_gray =  imagej_auto_contrast(f2_gray, 0.35)
    blue = np.abs(f1_gray.astype(np.int16) - f2_gray.astype(np.int16)).astype(np.uint8)
    merged = np.stack([blue, f2_gray, f1_gray], axis=2)
    writer.append_data(merged)

writer.close()

In [ ]:
import imageio
import numpy as np

video1_path = r"F:\In Vivo XROMM EMG\28MAY2025\28MAY2025_raisin_trial4_rat101_C001H001S0001.avi"
video2_path = r"F:\In Vivo XROMM EMG\28MAY2025\28MAY2025_raisin_trial4_rat101_C002H001S0001.avi"
output_path = r'F:\In Vivo XROMM EMG\28MAY2025\Composite Videos\28MAY2025_raisin_trial4_rat101_cams1and2.avi'

reader1 = imageio.get_reader(video1_path)
reader2 = imageio.get_reader(video2_path)
meta1 = reader1.get_meta_data()
meta2 = reader2.get_meta_data()
fps = meta1['fps']  # Or min(meta1['fps'], meta2['fps'])

# Find the smallest number of frames
nframes = min(meta1['nframes'], meta2['nframes'])

# Determine output size (pad heights if needed)
height = max(meta1['size'][1], meta2['size'][1])
width = meta1['size'][0] + meta2['size'][0]

def pad_to_height(frame, height):
    if frame.shape[0] == height:
        return frame
    pad = height - frame.shape[0]
    top = pad // 2
    bot = pad - top
    return np.pad(frame, ((top, bot), (0,0), (0,0)), mode='constant')

writer = imageio.get_writer(output_path, fps=fps)

for i, (f1, f2) in enumerate(zip(reader1, reader2)):
    if i >= nframes:
        break
    # Pad if necessary
    f1 = pad_to_height(f1, height)
    f2 = pad_to_height(f2, height)
    # Concatenate along width axis (side by side)
    merged = np.concatenate((f1, f2), axis=1)
    writer.append_data(merged)
    if i % 50 == 0:
        print(f'Processed frame {i}', end='\r')

reader1.close()
reader2.close()
writer.close()
print("\nDone!")

In [ ]:

os.chdir(r'D:\XROMM DLC Networks')

task = "108_Feeding_RGB" # Enter the name of your experiment Task
experimenter = "Brocklehurst" # Enter the name of the experimenter
video = [
    r"F:\In vivo XROMM\28OCT24\Trials\28OCT24_trial1_rat108_raisin_rgb.avi",
    r"F:\In vivo XROMM\28OCT24\Trials\28OCT24_trial3_rat108_raisin_rgb.avi",
    r"F:\In vivo XROMM\29OCT24\Trials\29OCT24_rat108_trial1_brazil_rgb.avi",
    r"F:\In vivo XROMM\29OCT24\Trials\29OCT24_rat108_trial2_brazil_rgb.avi"
] # Enter the paths of your videos OR FOLDER you want to grab frames from.

path_config_file = deeplabcut.create_new_project(
    task,
    experimenter,
    video,
    copy_videos=True,
)

# NOTE: The function returns the path, where your project is. 


In [5]:
get_bodyparts_from_xma(r"D:\XROMM Trial Images\28OCT24\28OCT24_rat108_Trial01_2D_Points.csv", mode = 'rgb')

['marker001_cam1',
 'marker001_cam2',
 'marker002_cam1',
 'marker002_cam2',
 'marker003_cam1',
 'marker003_cam2',
 'marker004_cam1',
 'marker004_cam2',
 'marker005_cam1',
 'marker005_cam2',
 'marker006_cam1',
 'marker006_cam2',
 'marker007_cam1',
 'marker007_cam2',
 'marker008_cam1',
 'marker008_cam2',
 'marker009_cam1',
 'marker009_cam2',
 'marker010_cam1',
 'marker010_cam2',
 'marker011_cam1',
 'marker011_cam2',
 'marker012_cam1',
 'marker012_cam2',
 'marker013_cam1',
 'marker013_cam2',
 'marker014_cam1',
 'marker014_cam2',
 'marker015_cam1',
 'marker015_cam2',
 'marker016_cam1',
 'marker016_cam2',
 'marker017_cam1',
 'marker017_cam2',
 'marker018_cam1',
 'marker018_cam2',
 'marker019_cam1',
 'marker019_cam2']

In [2]:
os.chdir(r'D:\XROMM DLC Networks')
# You could also enter this manually (e.g. if the project is already created and you want to pick up, where you stopped...)
# Enter the path of the config file that was just created from the above step (check the folder):
path_config_file = r"D:\XROMM DLC Networks\108_Feeding_RGB-Brocklehurst-2025-09-09\config.yaml"

In [ ]:
%matplotlib inline
#AUTOMATIC:
deeplabcut.extract_frames(path_config_file, algo='kmeans', userfeedback=False)

In [ ]:
#def xma_to_dlc(path_config_file,data_path,dataset_name,scorer,nframes,nnetworks = 1):
config = path_config_file[:-12]
scorer = 'Brocklehurst'

#trialnames = [folder for folder in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, folder)) and not folder.startswith('.')]

### PART 1: Pick frames for dataset
#contents = os.listdir(data_path+"/"+trial)
#filename = [x for x in contents if ".csv" in x] # csv filename
TrialNames = ['29OCT24_rat108_trial1_brazil', '29OCT24_rat108_trial2_brazil', '28OCT24_trial1_rat108_raisin', '28OCT24_trial3_rat108_raisin']
TrialFolderNames = [x + '_rgb' for x in TrialNames]

for TrialName, TrialFolderName in zip(TrialNames, TrialFolderNames):

    df1 = pd.read_csv(os.path.join(config, 'labeled-data', TrialFolderName, TrialName + '_2D_Points.csv'))

    # read pointnames from header row
    pointnames = df1.columns[::2].astype(str).str[:-2].tolist()
    #pnames.append(pointnames)

    #if pointnames aren't the same across trials
    #if any(pnames[0] != x for x in pnames):
    #    raise ValueError('Make sure point names are consistent across trials')

    ### Part 2: Extract images and 2D point data

    relnames = []
    data = pd.DataFrame()
    # new training dataset folder
    newpath = os.path.join(config, 'labeled-data', TrialFolderName)
    h5_save_path = newpath+"/CollectedData_"+scorer+".h5"
    csv_save_path = newpath+"/CollectedData_"+scorer+".csv"

    relpath = os.path.join('labeled-data', TrialFolderName)

    img_list = list(pathlib.Path(newpath).glob('*.png'))

    img_numbers = []

    for i in img_list:
        relname = os.path.join(relpath, os.path.basename(i))
        relnames = relnames + [relname]
        
        img_name = os.path.basename(i)
        img_number = int(re.findall(r'\d+', img_name)[0])
        img_number = img_number
        img_numbers.append(img_number)

    # extract 2D points data
    #df1= dfs[trialnum]
    xpos1 = df1.iloc[img_numbers,0+(1-1)*2::4]
    ypos1 = df1.iloc[img_numbers,1+(1-1)*2::4]

    xpos2 = df1.iloc[img_numbers,0+(2-1)*2::4]
    #xpos2 += 1024
    ypos2 = df1.iloc[img_numbers,1+(2-1)*2::4]

    temp_data1 = pd.concat([xpos1,ypos1,xpos2,ypos2],axis=1).sort_index(axis=1)
    temp_data1.columns = range(temp_data1.shape[1])
    data = pd.concat([data,temp_data1])


    ### Part 3: Complete final structure of datafiles
    dataFrame = pd.DataFrame()
    temp = np.empty((data.shape[0],2,))
    temp[:] = np.nan
    for i,bodypart in enumerate(pointnames):
        index = pd.MultiIndex.from_product([[scorer], [bodypart], ['x', 'y']],names=['scorer', 'bodyparts', 'coords'])
        frame = pd.DataFrame(temp, columns = index, index = relnames)
        frame.iloc[:,0:2] = data.iloc[:, 2*i:2*i+2].values.astype(float)
        dataFrame = pd.concat([dataFrame, frame],axis=1)
    dataFrame.replace('', np.nan, inplace=True)
    dataFrame.replace(' NaN', np.nan, inplace=True)
    dataFrame.replace(' NaN ', np.nan, inplace=True)
    dataFrame.replace('NaN ', np.nan, inplace=True)
    dataFrame.apply(pd.to_numeric)
    dataFrame.to_hdf(h5_save_path, key="df_with_missing", mode="w")
    dataFrame.to_csv(csv_save_path,na_rep='NaN')
    print("...done.")

print("Training data extracted to projectpath/labeled-data. Now use deeplabcut.create_training_dataset")

In [ ]:
deeplabcut.check_labels(path_config_file, draw_skeleton = False) # this creates a subdirectory with the frames + your labels

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

In [ ]:
deeplabcut.train_network(
    path_config_file,
    shuffle=1,
    trainingsetindex=0,
    device="cuda:0",
    max_snapshots_to_keep=5,
    displayiters=10,
    save_epochs=5,
    epochs=200,
)

In [ ]:
deeplabcut.evaluate_network(path_config_file, Shuffles=[1], plotting=True)

In [ ]:
camdata_dir = r"D:\XROMM DLC Networks\108_Feeding_RGB-Brocklehurst-2025-09-09\videos-for-analysis"
videos_2_analyse = glob.glob(camdata_dir + '/*.avi')

deeplabcut.analyze_videos(
    path_config_file, 
    videos_2_analyse,
    device="cuda:0",
    save_as_csv=True
)

In [ ]:
camdata_dir = r"D:\XROMM DLC Networks\108_Feeding_RGB-Brocklehurst-2025-09-09\videos-for-analysis"
DLC_outs = glob.glob(camdata_dir + '/*DLC*.csv')

for i in  DLC_outs:
    camdata_file_base = os.path.basename(i)
    camdata_file_name = camdata_file_base.split('DLC')[0] + 'Predicted-Points-2D.csv'

    camdata = pd.read_csv(i, sep=',',header=None)
    pointnames = list(camdata.loc[1,1:].unique())

    # reformat CSV / get rid of headers
    camdata = camdata.loc[3:,1:]
    camdata.columns = range(camdata.shape[1])
    camdata.index = range(camdata.shape[0])

    # make new column names
    nvar = len(pointnames)
    pointnames = [item for item in pointnames for repetitions in range(2)]
    post = ["_X", "_Y"]*nvar
    cols = [m+str(n) for m,n in zip(pointnames,post)]

    # remove likelihood columns
    camdata = camdata.drop(camdata.columns[2::3],axis=1)
    camdata = camdata.astype('float')
    # replace col names with new indices

    c1dataX = camdata.iloc[:, ::4]
    #c1dataX[c1dataX > 1024] = c1dataX[c1dataX > 1024] - 1024    
    c1dataY = camdata.iloc[:, 1::4]
    c2dataX = camdata.iloc[:, 2::4]
    #c2dataX[c2dataX < 1024] = c2dataX[c2dataX < 1024] + 1024    
    #c2dataX = c2dataX.sub(1024)
    c2dataY = camdata.iloc[:, 3::4]

    data_all = (pd.concat([c1dataX, c1dataY, c2dataX, c2dataY],axis=1).sort_index(axis=1))
    data_all.columns = cols
    data_all[data_all < 0] = 1
    #data_all.to_hdf(h5_save_path, key="df_with_missing", mode="w")
    data_all.to_csv(camdata_dir + '/' + camdata_file_name,na_rep='NaN',index=False)
    #c2cols = list(range(2,camdata.shape[1],4)) + list(range(3,camdata.shape[1],4))


In [ ]:
camdata_dir = r"D:\XROMM DLC Networks\108_Feeding_RGB-Brocklehurst-2025-09-09\videos-for-analysis"
videos_2_analyse = glob.glob(camdata_dir + '/*.avi')

deeplabcut.extract_outlier_frames(path_config_file, videos_2_analyse[0], automatic = True, outlieralgorithm="uncertain", p_bound  = 0.05)

In [ ]:
# ADD XMA TO DLC CODE CHUNK 

In [ ]:
deeplabcut.merge_datasets(config_path)

In [ ]:
#h5_save_path = savepath+"/"+trialname+"-Predicted2DPoints.h5"
#csv_save_path = savepath+"/"+trialname+"-Predicted2DPoints.csv"

cam12data = pd.read_csv(cam12data_file, sep=',',header=None)
pointnames = list(cam12data.loc[1,1:].unique())

# reformat CSV / get rid of headers
cam12data = cam12data.loc[3:,1:]
cam12data.columns = range(cam12data.shape[1])
cam12data.index = range(cam12data.shape[0])

# make new column names
nvar = len(pointnames)
pointnames = [item for item in pointnames for repetitions in range(2)]
post = ["_X", "_Y"]*nvar
cols = [m+str(n) for m,n in zip(pointnames,post)]

 # remove likelihood columns
cam12data = cam12data.drop(cam12data.columns[2::3],axis=1)
cam12data = cam12data.astype('float32')
# replace col names with new indices

c1dataX = cam12data.iloc[:, ::4]
c1dataY = cam12data.iloc[:, 1::4]
c2dataX = cam12data.iloc[:, 2::4]
c2dataX = c2dataX.sub(1024)
c2dataY = cam12data.iloc[:, 3::4]

data_all = (pd.concat([c1dataX, c1dataY, c2dataX, c2dataY],axis=1).sort_index(axis=1))
data_all.columns = cols
data_all[data_all < 0] = 1
#data_all.to_hdf(h5_save_path, key="df_with_missing", mode="w")
data_all.to_csv('28MAY2025_raisin_trial4_rat101_cams1and2_Predicted-Points.csv',na_rep='NaN',index=False)
#c2cols = list(range(2,cam12data.shape[1],4)) + list(range(3,cam12data.shape[1],4))

In [ ]:


   

    # replace col names with new indices
    c1cols = list(range(0,cam12data.shape[1]*2,4)) + list(range(0,1 + cam12data.shape[1]*2,4))
    c2cols = list(range(2,cam12data.shape[1]*2,4)) + list(range(3,cam12data.shape[1]*2,4))
    c1cols.sort()
    c2cols.sort()
    cam1data.columns = c1cols
    cam2data.columns = c2cols

    df = pd.concat([cam1data,cam2data],axis=1).sort_index(axis=1)
    df.columns = cols
    df.to_hdf(h5_save_path, key="df_with_missing", mode="w")
    df.to_csv(csv_save_path,na_rep='NaN',index=False)


In [ ]:

deeplabcut.check_labels(path_config_file)